In [2]:
pip install boto3


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [228]:
import boto3
import email
client = boto3.client('textract')

In [229]:
client

In [230]:
s3 = boto3.client('s3')
bucket_name = 'infiniteloopbucket'
object_key = 'Category - Cold emails/cold email example to mark cuban.png'

response = s3.get_object(Bucket=bucket_name, Key=object_key)
image_bytes = response['Body'].read()
attachment = email.message_from_bytes(image_bytes)

In [231]:
attachment

In [232]:
image_bytes = attachment.get_payload(decode=True)

In [233]:
response = client.detect_document_text(Document={'Bytes': image_bytes})

In [244]:
sumConfidence = 0 
lenBlk = 0
for block in response['Blocks']:
    if block['BlockType'] == 'LINE':
        #print(block['Text'], ' Confidence: ', block['Confidence'])
        sumConfidence += block['Confidence']
        lenBlk += 1
avgConfidence = sumConfidence/lenBlk
avgConfidence

99.0411124596229

In [234]:
text = ''
for item in response['Blocks']:
    if item['BlockType'] == 'LINE':
        text += item['Text'] + '\n'

print(text)

Reaching out - Factmata
Dear Mr. Cuban,
Apologies for my cold message. / am the founder of a Google backed startup called
Factmata, that uses artificial intelligence to perform automated fact checking and
referencing. We are a team of 3 NLP researchers and scientists with 30+ published
and cited papers within natural language processing, question answering and
information extraction.l am currently fundraising from people who care about the
problem of online misinformation, want to reduce mistrust in the media, and change
the way we consume online content. / would love to tell you more about us if of
interest, especially given your recent public discussions about this topic.
Look forward to hearing from you soon,
Best regards,
Dhruv



In [217]:
import pandas as pd
df = pd.DataFrame(columns=['email_text', 'label'])

In [218]:
df

,email_text,label


In [193]:
objects[0]['Key']

'.write_access_check_file.temp'

In [194]:
#objects in s3 includes folders and image objects
objects = s3.list_objects_v2(Bucket=bucket_name)['Contents']

In [195]:
objKey = objects[3]['Key']

In [196]:
response = s3.get_object(Bucket=bucket_name, Key=objKey)
image_bytes = response['Body'].read()
attachment = email.message_from_bytes(image_bytes)
image_bytes = attachment.get_payload(decode=True)
responseEmail = client.detect_document_text(Document={'Bytes': image_bytes})
text = '\n'.join([item['Text'] for item in responseEmail['Blocks'] if item['BlockType'] == 'LINE'])

In [197]:
text
label = objKey.split('/')[0].split(' - ')[1]

In [198]:
label

'Cold emails'

In [220]:
for obj in objects:
    objKey = obj['Key'] #key is either the folder key or the image object key
    
    # Skip objects that are not images
    if not objKey.endswith('.jpg') and not objKey.endswith('.jpeg') and not objKey.endswith('.png'):
        continue
        
    # Extract the image bytes from the S3 object
    response = s3.get_object(Bucket=bucket_name, Key=objKey)
    image_bytes = response['Body'].read()
    attachment = email.message_from_bytes(image_bytes)
    image_bytes = attachment.get_payload(decode=True)
    responseEmail = client.detect_document_text(Document={'Bytes': image_bytes})
    text = '\n'.join([item['Text'] for item in responseEmail['Blocks'] if item['BlockType'] == 'LINE'])
    label = objKey.split('/')[0].split(' - ')[1]
    df = df.append({'email_text': text, 'label':label}, ignore_index=True)
    
    #print(f'Text extracted from object {objKey}:')
    #print(text)

In [221]:
df

,email_text,label
0,"Reaching out - Factmata\nDear Mr. Cuban,\nApol...",Cold emails
1,John Lee (MIT graduate) - Inquiry into Biomedi...,Cold emails
2,Editorial internship inquiry\nCarolina VonKamp...,Cold emails
3,Interested in Eight Sleep Marketing Internship...,Cold emails
4,"Hey, my name's Niraj Pant.\nI understand your ...",Cold emails
...,...,...
206,"To:\nFrom:\nSubject:\nMs. Decision Maker,\nI a...",interview acceptance
207,The following template can be used for email i...,interview acceptance
208,To:\nFrom:\nSubject:\nMr./Ms. [Recruiter or Hi...,interview acceptance
209,"Dear Y,\nThank you very much for your mail.\nI...",interview acceptance


In [114]:
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
df1

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [222]:
df = df[['label', 'email_text']]

In [223]:
df = df.sample(frac = 1).reset_index(drop = True)

In [224]:
df.to_csv('HRemail.csv', index=False, header=False)

In [225]:
csv_string = df.to_csv(index=False)

In [226]:
s3.put_object(Bucket=bucket_name, Key='HRemail', Body=csv_string.encode('utf-8'))

{'ResponseMetadata': {'RequestId': 'S0Y5DG743AEBKZBD',
  'HostId': 'bJxmwGcDCERrJOqb03V7F5uUa0xT8x6hfpttZS8Pp0jiwUCicVBa8GquBmIehDQdVh+xzHNkUhU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'bJxmwGcDCERrJOqb03V7F5uUa0xT8x6hfpttZS8Pp0jiwUCicVBa8GquBmIehDQdVh+xzHNkUhU=',
   'x-amz-request-id': 'S0Y5DG743AEBKZBD',
   'date': 'Sat, 11 Mar 2023 17:09:54 GMT',
   'x-amz-version-id': 'n170JEGgvGytEnnbFGz8EPiSyvpDR8xy',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"43603ae7aab19168fdda44cf4986cea7"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"43603ae7aab19168fdda44cf4986cea7"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'n170JEGgvGytEnnbFGz8EPiSyvpDR8xy'}

In [148]:
obj = s3.get_object(Bucket=bucket_name, Key='HRemail.csv')
HRemail_df = pd.read_csv(obj['Body'])

In [143]:
HRemail_df.to_csv('HRemail_df.csv', index=False)

In [167]:
df[df['label']=='Cold emails']

,label,email_text
1,Cold emails,"Hello,\nI noticed your Twitter post about the ..."
6,Cold emails,Letter Accepting an Interview Invitation and A...
7,Cold emails,"Dear [NAME],\nMy name is [YOUR NAME]. You grad..."
8,Cold emails,"Hey Jennifer,\nI'm reaching out regarding your..."
10,Cold emails,Sample Message Confirming Date and Time When D...
...,...,...
671,Cold emails,"From:\nSubject: Quick favor\nDate: October 9, ..."
672,Cold emails,"Subject: Dave, You Could Conquer the Cubs\nHi ..."
673,Cold emails,A huge nerd looking for an internship at Nerds...
674,Cold emails,Berkeley Undergrad - Corporate Strategy and Bu...


In [169]:
df.shape

(708, 2)